In [1]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
from toy_plot_helpers import ToyPlot
plot = ToyPlot()

In [2]:
old_store = paths.storage.Storage("mstis_bootstrap.nc", "r")

A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

In [3]:
print "PathMovers:", len(old_store.pathmovers)
print "Samples:", len(old_store.samples)
print "Ensembles:", len(old_store.ensembles)
print "SampleSets:", len(old_store.samplesets)
print "Snapshots:", len(old_store.snapshots)
print "Networks:", len(old_store.networks)

PathMovers: 0
Samples: 12
Ensembles: 120
SampleSets: 1
Snapshots: 1246
Networks: 0


In [4]:
topol = old_store.topology

In [5]:
# TODO: current loading won't work unless I populate cache first (these lines do it)
# Does the new storage fix that?
print [cv.name for cv in old_store.collectivevariables]
print [vol.name for vol in old_store.volumes]

[u'opA', u'opB', u'opC']
[u'A', '0.0<opA<0.04', '[UnionVolume]', u'all states except A', u'B', u'C', '[UnionVolume]', '0.0<opA<0.09', '[UnionVolume]', '[UnionVolume]', '0.0<opA<0.16', '[UnionVolume]', '[UnionVolume]', '0.0<opA<0.25', '[UnionVolume]', '[UnionVolume]', '0.0<opB<0.04', '[UnionVolume]', u'all states except B', '[UnionVolume]', '0.0<opB<0.09', '[UnionVolume]', '[UnionVolume]', '0.0<opB<0.16', '[UnionVolume]', '[UnionVolume]', '0.0<opB<0.25', '[UnionVolume]', '[UnionVolume]', '0.0<opC<0.04', '[UnionVolume]', u'all states except C', '[UnionVolume]', '0.0<opC<0.09', '[UnionVolume]', '[UnionVolume]', '0.0<opC<0.16', '[UnionVolume]', '[UnionVolume]', '0.0<opC<0.25', '[UnionVolume]', '[UnionVolume]']


In [6]:
opA = old_store.collectivevariables.find('opA')[0]
opB = old_store.collectivevariables.find('opB')[0]
opC = old_store.collectivevariables.find('opC')[0]

In [7]:
stateA = old_store.volumes.find('A')[0]
stateB = old_store.volumes.find('B')[0]
stateC = old_store.volumes.find('C')[0]

In [8]:
# we could also load the interfaces, but it takes less code to build new ones:
interfacesA = paths.VolumeFactory.CVRangeVolumeSet(opA, 0.0,[0.2**2, 0.3**2, 0.4**2, 0.5**2])
interfacesB = paths.VolumeFactory.CVRangeVolumeSet(opB, 0.0,[0.2**2, 0.3**2, 0.4**2, 0.5**2])
interfacesC = paths.VolumeFactory.CVRangeVolumeSet(opC, 0.0,[0.2**2, 0.3**2, 0.4**2, 0.5**2])

In [9]:
mstis = paths.MSTISNetwork([
    (stateA, interfacesA, "A", opA),
    (stateB, interfacesB, "B", opB),
    (stateC, interfacesC, "C", opC)
])

Now we need to set up real trajectories that we can use for each of these.

**explain a sample**

First, even though the ensembles are semantically identical to what we have before, they aren't actually the same  objects, because they were created by building a new `MSTISNetwork`. That means that, even though our saved trajectories satisfy the correct ensembles, we still need a way to identify which of the new ensembles to associate them with.

There are two main ways to do this. The first is to take one trajectory, and associate it with as many ensembles as possible. If your first path comes from a TPS simulation, that is the approach you'll want to take.

The second approach is better suited to our conditions here: we already have a good trajectory for each ensemble. So we just want to remap our old ensembles to new ones.

In [10]:
# load the sample set we have saved before
old_sampleset = old_store.samplesets[0]

### Loading one trajectory into lots of ensembles

In [12]:
# this makes a dictionary mapping the outermost ensemble of each sampling transition 
# to a trajectory from the old_sampleset that satisfies that ensemble
trajs = {}
for ens in [t.ensembles[-1] for t in mstis.sampling_transitions]:
    trajs[ens] = [s.trajectory for s in old_sampleset if ens(s.trajectory)==True][0]
    
assert(len(trajs)==3) # otherwise, we have a problem

In [19]:
initial_samples = {}
for t in mstis.sampling_transitions:
    initial_samples[t] = paths.SampleSet.map_trajectory_to_ensembles(trajs[t.ensembles[-1]], t.ensembles)

In [23]:
for s in initial_samples.values()[0]:
    print s

Replica: 0
Trajectory: Trajectory[47]
Ensemble: <openpathsampling.ensemble.TISEnsemble object at 0x11a7aa690>

Replica: 1
Trajectory: Trajectory[47]
Ensemble: <openpathsampling.ensemble.TISEnsemble object at 0x11a7aaa90>

Replica: 2
Trajectory: Trajectory[47]
Ensemble: <openpathsampling.ensemble.TISEnsemble object at 0x11a7aae90>



In [25]:
sset = paths.SampleSet.relabel_replicas_per_ensemble(initial_samples.values())

In [26]:
len(sset)

9

### Remapping old ensembles to new ensembles

If your old and new ensembles have the same string representations, then OPS has a function to help you automatically map them. As long as you create the ensembles in the same way, they'll have the same string representation. Note that if you *don't* have the same string representation, you would have to assign trajectories to ensembles by hand (which isn't that hard, but is a bit tedious).

### Setting up special ensembles

#### Multiple state ensemble

#### Minus interface ensemble

## Equilibrating within the path ensemble

## Running RETIS